In [7]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.metrics import make_scorer, r2_score
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import pickle 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import ParameterGrid

import warnings
warnings.simplefilter(action='ignore', category=Warning)
# more

In [8]:
# df = pd.read_csv('factors_2002.csv', parse_dates=['DATE'])

In [9]:
# with open('data/features_1965.pkl', 'wb') as f:
#     pickle.dump(df, f)

with open('data/features_1965.pkl', 'rb') as f:
    df = pickle.load(f)
    print(df.head())



   permno       DATE        mvel1      beta    betasq     chmom     dolvol  \
0   10145 1965-02-26   1498872.00  0.983510  0.967291  0.105988  11.546907   
1   10401 1965-02-26  35392058.00  0.780829  0.609694 -0.063768  12.240330   
2   10786 1965-02-26   1695284.75  0.806119  0.649827 -0.130519  12.005040   
3   10989 1965-02-26   1295887.75  1.199748  1.439395  0.073609  11.756961   
4   11260 1965-02-26   2302001.25  1.257269  1.580725 -0.167320  12.240330   

    idiovol    indmom     mom1m  ...  macro_ep  macro_bm  macro_ntis  \
0  0.022307  0.035075  0.104116  ...  2.936836  0.471399    0.014823   
1  0.013395  0.335139 -0.007326  ...  2.936836  0.471399    0.014823   
2  0.024366  0.104106  0.060498  ...  2.936836  0.471399    0.014823   
3  0.022717  0.118513  0.068807  ...  2.936836  0.471399    0.014823   
4  0.035883  0.185424 -0.036885  ...  2.936836  0.471399    0.014823   

   macro_tbl  macro_tms  macro_dfy  macro_svar  macro_mkt-rf  macro_hml  \
0     0.0393    -0.0379

In [10]:
#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar,macro_mkt-rf,macro_hml,macro_smb
0,10145,1965-02-26,1498872.00,0.983510,0.967291,0.105988,11.546906,0.022307,0.035075,0.104116,...,2.936836,0.471399,0.014823,0.0393,-0.0379,0.0055,0.000393,0.44,0.11,3.55
1,10401,1965-02-26,35392056.00,0.780829,0.609694,-0.063768,12.240331,0.013395,0.335139,-0.007326,...,2.936836,0.471399,0.014823,0.0393,-0.0379,0.0055,0.000393,0.44,0.11,3.55
2,10786,1965-02-26,1695284.75,0.806119,0.649827,-0.130519,12.005040,0.024366,0.104106,0.060498,...,2.936836,0.471399,0.014823,0.0393,-0.0379,0.0055,0.000393,0.44,0.11,3.55
3,10989,1965-02-26,1295887.75,1.199748,1.439395,0.073609,11.756961,0.022717,0.118513,0.068807,...,2.936836,0.471399,0.014823,0.0393,-0.0379,0.0055,0.000393,0.44,0.11,3.55
4,11260,1965-02-26,2302001.25,1.257269,1.580725,-0.167320,12.240331,0.035883,0.185424,-0.036885,...,2.936836,0.471399,0.014823,0.0393,-0.0379,0.0055,0.000393,0.44,0.11,3.55


In [11]:
df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [12]:
p=0.3 
df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


In [16]:
features = df.columns[~df.columns.isin(['DATE', 'DATE2', "mvel2",'sic2' ,'permno',"permno2",'risk_premium'])].tolist()
df[features]=df.groupby('DATE')[features].rank(pct=True)
df[features]= 2*df[features] -1

df_large[features]=df_large.groupby("DATE")[features].rank(pct=True)
df_large[features]= 2*df_large[features] -1

df_small[features]=df_small.groupby("DATE")[features].rank(pct=True)
df_small[features]= 2*df_small[features] -1

In [17]:
#Define huber loss function to minimize in the validation step
def huber_loss_error(y_true, y_pred, delta=1.35):
    res = []
    for i in zip(y_true, y_pred):
        if abs(i[0]-i[1])<=delta:
            res.append(0.5*((i[0]-i[1])**2))
        else:
            res.append(delta*((abs(i[0]-i[1]) )-0.5*(delta**2)))
    return np.sum(res)

In [20]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['DATE', 'DATE2', "mvel2",'sic2' ,'permno',"permno2",'risk_premium'])].tolist()

X = df[features]
y = df[['risk_premium']]

predictions = []
y_test_list =[]
dates = []
dic_r2_all = {}

dic_max_depth_all = {}

# Grid of prespecified values for each hyperparameter  
param_grid = {'max_depth': [1,2,3,4,5,6,7,8], 
              'n_estimators': [100], 
              "learning_rate": [0.01,0.1], 
              "max_features": ["sqrt"],
               "min_samples_split" : [5000,8000,10000],
               "min_samples_leaf": [50,100,200]}
#convert grid to list containing all posible hyperparameter combinations to iterate over
grid = list(ParameterGrid(param_grid))
# Empty container to save the objective loss function (huber loss error) for each hyperparameter combination
huber_loss = np.full((len(grid),1),np.nan, dtype = np.float32)

for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(1981,1,31), second_split_date= datetime.date(1991,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]
    
    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]
    
    #Loop over the list containing all pyperparameter combinations, fit on the training sample and use 
    #validation set to generate predictions
    for i in range(len(grid)):
        GBR_val = GradientBoostingRegressor(loss='huber',
                                            max_depth=grid[i]["max_depth"], 
                                            learning_rate=grid[i]["learning_rate"],
                                            n_estimators = grid[i]["n_estimators"],
                                            max_features= grid[i]["max_features"],
                                            min_samples_split= grid[i]["min_samples_split"],
                                            min_samples_leaf= grid[i]["min_samples_leaf"])
    
        GBR_val.fit(X_train, y_train)
        Yval_predict=GBR_val.predict(X_val)
        #calculate huber loss for each hyperparameter combination
        #set delta to 1.35 following Huber (2004)
        huber_loss[i,0]= huber_loss_error(y_val,Yval_predict, delta=1.35)


    #The optimal combination of hyperparameters is the one that causes the lowest loss 
    optim_param = grid[np.argmin(huber_loss)]

    
    #Fit again using the train and validation set and the optimal value for each hyperparameter 
    GBR = GradientBoostingRegressor(loss='huber',
                                    max_depth=optim_param["max_depth"], 
                                    learning_rate=optim_param["learning_rate"],
                                    n_estimators = optim_param["n_estimators"],
                                    max_features= optim_param["max_features"],
                                    min_samples_split= optim_param["min_samples_split"],
                                    min_samples_leaf= optim_param["min_samples_leaf"])

    GBR.fit(np.concatenate((X_train, X_val)), np.concatenate((y_train, y_val)))
    preds=GBR.predict(X_test)
    
    #Save predictions, dates and the true values of the dependent variable to list
    predictions.append(preds)
    dates.append(y_test.index)
    y_test_list.append(y_test)

    #Calculate OOS model performance the for current window
    r2 = 1-np.sum(pow(y_test-preds,2))/np.sum(pow(y_test,2))
    #Save OOS model performance and the respective month to dictionary
    dic_r2_all["r2." + str(y_test.index)] = r2
    # Save the number of predictors randomly considered as potential split variables
    dic_max_depth_all["feat." + str(y_test.index)]= optim_param["max_depth"]
   
    
    
#Concatenate to get results over the whole OOS test period (Jan 2010-Dec 2019)
predictions_all= np.concatenate(predictions, axis=0)
y_test_list_all= np.concatenate(y_test_list, axis=0) 
dates_all= np.concatenate(dates, axis=0)

#Calculate OOS model performance over the entire test period in line with Gu et al (2020)
R2OOS_GBR = r2_score(y_list_list_all, predictions_all)
print("R2OOS gradient boosted regression tree: ", R2OOS_GBR)

KeyError: 'DATE'